# Reading The Liberty File

In [2]:
import sys
%pwd

'c:\\Users\\Amik\\research\\rtl_synthesis\\synthesized_verilog_parser\\note_books'

In [3]:
# File Path:
file_path = "../data/liberty_file/tsmc_180/slow.lib"
output_file_path =  "../data/outputs/parsed_library_file.json"

def read_and_return_liberty_file():
    with open(file_path, 'r') as file:
        data = file.read()
    return data

# Getting The Gate Names and Dumping Into a .txt File

In [4]:
import re
#output_file_path = "../data/outputs/unique_gate_names.txt"


def extract_gate_names_from_liberty(liberty_data):
    pattern = r"cell\s+\(([^)]+)\)"
    gate_names = re.findall(pattern, liberty_data)
    # To get unique gate names
    unique_gate_names = list(set(gate_names))
    return unique_gate_names

def dump_gate_names_to_txt(gate_names):
    with open(output_file_path, 'w') as file:
        for name in gate_names:
            file.write(name + '\n')

def read_gate_names_from_txt():
    with open(output_file_path, 'r') as file:
        gate_names = [line.strip() for line in file.readlines()]
    return gate_names

liberty_data = read_and_return_liberty_file()


print((liberty_data[0]))
unique_gate_names = extract_gate_names_from_liberty(liberty_data)
dump_gate_names_to_txt(unique_gate_names)

# To read the gate names from the txt file
gate_names_from_txt = read_gate_names_from_txt()



/


In [5]:
def gate_counter_from_boolean_expression(expression):
    
    gates = ['^', '|', '!']
    total_gates = 0

    i = 0
    while i < len(expression):
        char = expression[i]
        
        if char == ' ':
            
            if i > 0 and i < len(expression) - 1:
                
                # Check for the two specified cases
                if (expression[i-1] == ')' and expression[i+1] == '('):
                    expression = expression[:i] + '^' + expression[i+1:]
                if (expression[i-1] == ')' and expression[i+1].isalnum()):
                    expression = expression[:i] + '^' + expression[i+1:]
                if (expression[i-1].isalnum() and expression[i+1] == '('):
                    expression = expression[:i] + '^' + expression[i+1:]
                if (expression[i-1].isalnum() and expression[i+1].isalnum()):                   
                    expression = expression[:i] + '^' + expression[i+1:]
        i += 1
    
    # Construct the regex pattern
    pattern = '[' + re.escape(''.join(gates)) + ']'

    # Find all occurrences of the gates in the expression
    total_gates = len(re.findall(pattern, expression))
    return total_gates




def extract_block_area_and_cell_leagage_power(block):
    matches = re.search(r"cell_footprint : (.*?);[\s\S]*?area : (.*?);[\s\S]*?cell_leakage_power : (.*?);", block)
    
    if matches:
        cell_footprint = matches.group(1)
        area = matches.group(2)
        cell_leakage_power = matches.group(3)
        return cell_footprint, area, cell_leakage_power
    else:
        return 0, 0, 0
    
    
    

def extract_pin_name(pin):
    pin_name_pattern = r"^(.*?)\)"

    # Search for the pin name
    pin_name_match = re.search(pin_name_pattern, pin, re.MULTILINE)

    # Extract the pin name
    if pin_name_match:
        pin_name = pin_name_match.group(1).strip()
        return pin_name
    else:
        print("Pin name not found.")
        sys.exit()

        
        

def extract_block_name(block):
    # Define the regular expression pattern to match the block name
    block_name_pattern = r"^(.*?)\)"

    # Search for the block name
    block_name_match = re.search(block_name_pattern, block, re.MULTILINE)

    # Extract the block name
    if block_name_match:
        block_name = block_name_match.group(1).strip()
        return block_name
    else:
        print("Block name not found.")
        sys.exit()

    
    
def extract_pin_information(block):
    pins = block.split("pin(")
    
    pin_dict = {}
    total_number_of_gates_in_block = 0
    
    for pin in pins[1:]:
        
        pin_name = extract_pin_name(pin)
        
        
        # Match each parameter separately
        direction_match = re.search(r"direction : (.*?);", pin)
        capacitance_match = re.search(r"capacitance : (.*?);", pin)
        function_match = re.search(r"function : \"(.*?)\";", pin)
        max_capacitance_match = re.search(r"max_capacitance : (.*?);", pin)

      
        pin_information = {}

        # Add direction if available
        if direction_match:
            pin_information["direction"] = direction_match.group(1)

        # Add capacitance if available
        if capacitance_match:
            pin_information["capacitance"] = float(capacitance_match.group(1))

        # Add function if available
        if function_match:
            pin_information["function"] = function_match.group(1)
            number_of_gates = gate_counter_from_boolean_expression(pin_information["function"])
            pin_information["number_of_gates"] = number_of_gates
            
            total_number_of_gates_in_block += number_of_gates
        # Add max_capacitance if available
        if max_capacitance_match:
            pin_information["max_capacitance"] = float(max_capacitance_match.group(1))
          
        
        # final pin dict
        pin_dict[pin_name] = pin_information 
    
    pin_dict['total_number_of_gates_in_block'] = total_number_of_gates_in_block
        
    return pin_dict
        

        
        
        
        

def parse_liberty_file(liberty_content):
    # Initialize an empty dictionary to store the library information
    lib_dictionary = {}

    # Split the content into blocks based on "cell (" as a separator
    blocks = liberty_content.split("cell (")
    
    
    block_dict = {}
    
    # Iterate through the blocks to extract block information
    for block in blocks[1:]:  # Skip the first empty block
        
        block_name = extract_block_name(block)
        block_dict[block_name] = {}
        
        
        cell_footprint, area, cell_leakage_power = extract_block_area_and_cell_leagage_power(block)
        block_dict[block_name]['cell_footprint'] = cell_footprint
        block_dict[block_name]['area'] = area
        block_dict[block_name]['cell_leakage_power'] = cell_leakage_power
        
        
        pin_dict = extract_pin_information(block)
        block_dict[block_name]['pins'] = pin_dict
        block_dict[block_name]['total_number_of_gates_in_block'] = block_dict[block_name]['pins']['total_number_of_gates_in_block']
        del block_dict[block_name]['pins']['total_number_of_gates_in_block']
        
        
        
    return block_dict


# Read the content of the liberty file
liberty_content = read_and_return_liberty_file()

# Parse the liberty content into a dictionary
lib_dictionary = parse_liberty_file(liberty_content)

# Print the resulting dictionary
print(lib_dictionary)


{'ADDFHX1': {'cell_footprint': 'addfh', 'area': '76.507200', 'cell_leakage_power': '4104.182520', 'pins': {'A': {'direction': 'input', 'capacitance': 0.006053}, 'B': {'direction': 'input', 'capacitance': 0.013876}, 'CI': {'direction': 'input', 'capacitance': 0.004189}, 'S': {'direction': 'output', 'capacitance': 0.0, 'function': '(A ^ B ^ CI)', 'number_of_gates': 2, 'max_capacitance': 0.3115}, 'CO': {'direction': 'output', 'capacitance': 0.0, 'function': '(((A ^ B) CI) | (A B))', 'number_of_gates': 4, 'max_capacitance': 0.3115}}, 'total_number_of_gates_in_block': 6}, 'ADDFHX2': {'cell_footprint': 'addfh', 'area': '113.097600', 'cell_leakage_power': '7522.904160', 'pins': {'A': {'direction': 'input', 'capacitance': 0.011305}, 'B': {'direction': 'input', 'capacitance': 0.025038}, 'CI': {'direction': 'input', 'capacitance': 0.007735}, 'S': {'direction': 'output', 'capacitance': 0.0, 'function': '(A ^ B ^ CI)', 'number_of_gates': 2, 'max_capacitance': 0.623}, 'CO': {'direction': 'output', 

In [6]:
import json

def save_dict_to_file(lib_dictionary, output_file_path):
    with open(output_file_path, 'w') as file:
        json.dump(lib_dictionary, file)
        
        
save_dict_to_file(lib_dictionary, output_file_path)


In [7]:
def load_dict_from_file(output_file_path):
    with open(output_file_path, 'r') as file:
        return json.load(file)

# Example usage
loaded_dict = load_dict_from_file(output_file_path)
print(loaded_dict)


{'ADDFHX1': {'cell_footprint': 'addfh', 'area': '76.507200', 'cell_leakage_power': '4104.182520', 'pins': {'A': {'direction': 'input', 'capacitance': 0.006053}, 'B': {'direction': 'input', 'capacitance': 0.013876}, 'CI': {'direction': 'input', 'capacitance': 0.004189}, 'S': {'direction': 'output', 'capacitance': 0.0, 'function': '(A ^ B ^ CI)', 'number_of_gates': 2, 'max_capacitance': 0.3115}, 'CO': {'direction': 'output', 'capacitance': 0.0, 'function': '(((A ^ B) CI) | (A B))', 'number_of_gates': 4, 'max_capacitance': 0.3115}}, 'total_number_of_gates_in_block': 6}, 'ADDFHX2': {'cell_footprint': 'addfh', 'area': '113.097600', 'cell_leakage_power': '7522.904160', 'pins': {'A': {'direction': 'input', 'capacitance': 0.011305}, 'B': {'direction': 'input', 'capacitance': 0.025038}, 'CI': {'direction': 'input', 'capacitance': 0.007735}, 'S': {'direction': 'output', 'capacitance': 0.0, 'function': '(A ^ B ^ CI)', 'number_of_gates': 2, 'max_capacitance': 0.623}, 'CO': {'direction': 'output', 

In [8]:
import re


